# Deploy Llama 2 (70B) on Amazon SageMaker

LLaMA 2 is the next version of the LLaMA. It is trained on more data - 2T tokens and supports context length window upto 4K tokens. Meta fine-tuned conversational models with Reinforcement Learning from Human Feedback on over 1 million human annotations.

In this notebook you will learn how to deploy Llama 2 model to Amazon SageMaker. We are going to use the Hugging Face LLM DLC is a new purpose-built Inference Container to easily deploy LLMs in a secure and managed environment. The DLC is powered by [Text Generation Inference (TGI)](https://aws.amazon.com/blogs/machine-learning/announcing-the-launch-of-new-hugging-face-llm-inference-containers-on-amazon-sagemaker/) a scalelable, optimized solution for deploying and serving Large Language Models (LLMs).


## 1. Setup development environment

We are going to use the `sagemaker` python SDK to deploy Llama 2 to Amazon SageMaker. We need to make sure to have an AWS account configured and the `sagemaker` python SDK installed. 

In [ ]:
!pip install "sagemaker>=2.175.0" --upgrade --quiet

If you are going to use Sagemaker in a local environment. You need access to an IAM Role with the required permissions for Sagemaker. You can find [here](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-roles.html) more about it.


In [ ]:
import sagemaker
import boto3
sess = sagemaker.Session()
# sagemaker session bucket -> used for uploading data, models and logs
# sagemaker will automatically create this bucket if it not exists
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

print(f"sagemaker role arn: {role}")
print(f"sagemaker session region: {sess.boto_region_name}")


## 2. Retrieve the new Hugging Face LLM DLC

Compared to deploying regular Hugging Face models we first need to retrieve the container uri and provide it to our `HuggingFaceModel` model class with a `image_uri` pointing to the image. To retrieve the new Hugging Face LLM DLC in Amazon SageMaker, we can use the `get_huggingface_llm_image_uri` method provided by the `sagemaker` SDK. This method allows us to retrieve the URI for the desired Hugging Face LLM DLC based on the specified `backend`, `session`, `region`, and `version`. You can find the available versions [here](https://github.com/aws/deep-learning-containers/blob/master/available_images.md#huggingface-text-generation-inference-containers)


In [ ]:
from sagemaker.huggingface import get_huggingface_llm_image_uri

# retrieve the llm image uri
llm_image = get_huggingface_llm_image_uri(
  "huggingface",
  version="0.9.3"
)

# print ecr image uri
print(f"llm image uri: {llm_image}")

## 3. Deploy Llama 2 to Amazon SageMaker

In [ ]:
import json
from sagemaker.huggingface import HuggingFaceModel

# sagemaker config
instance_type = "ml.p4d.24xlarge"
number_of_gpu = 8
health_check_timeout = 300

# Define Model and Endpoint configuration parameter
config = {
  'HF_MODEL_ID': "meta-llama/Llama-2-70b-chat-hf", # model_id from hf.co/models
  'SM_NUM_GPUS': json.dumps(number_of_gpu), # Number of GPU used per replica
  'MAX_INPUT_LENGTH': json.dumps(2048),  # Max length of input text
  'MAX_TOTAL_TOKENS': json.dumps(4096),  # Max length of the generation (including input text)
  'MAX_BATCH_TOTAL_TOKENS': json.dumps(8192),  # Limits the number of tokens that can be processed in parallel during the generation
  'HUGGING_FACE_HUB_TOKEN': <REPLACE WITH YOUR TOKEN>,
}

# check if token is set
assert config['HUGGING_FACE_HUB_TOKEN'] != "<REPLACE WITH YOUR TOKEN>", "Please set your Hugging Face Hub token"

# create HuggingFaceModel with the image uri
llm_model = HuggingFaceModel(
  role=role,
  image_uri=llm_image,
  env=config
)

After we have created the `HuggingFaceModel` we can deploy it to Amazon SageMaker using the `deploy` method. We will deploy the model with the `ml.p4d.24xlarge` instance type. TGI will automatically distribute and shard the model across all GPUs.

In [ ]:
# Deploy model to an endpoint
# https://sagemaker.readthedocs.io/en/stable/api/inference/model.html#sagemaker.model.Model.deploy
llm = llm_model.deploy(
  initial_instance_count=1,
  instance_type=instance_type,
  container_startup_health_check_timeout=health_check_timeout, # 10 minutes to be able to load the model
)


In [ ]:

prompt = "write a program to add two numbers in python"
#fib_prompt = "write a program to create a fibonacci sequence in python"
for i in range(5):
    chat = llm.predict({"inputs":prompt,
                        "parameters": {
                                    "max_new_tokens": 500,
                                    #"min_new_tokens": 500,
                                    "do_sample": True,
                                    "temperature": 0.4,
                                    "return_full_text": False
                            }
                       }
                      )

    print(chat)

In [ ]:
tokens_2k = """
Summarize the following:

Artificial Intelligence (AI) has emerged as a revolutionary force in the field of healthcare, reshaping the entire landscape of this vital industry. Its transformative effects span a wide spectrum, from enhancing medical diagnoses to streamlining administrative tasks, redefining treatment approaches, and even accelerating drug discovery. AI's prowess in medical diagnosis is particularly striking, with machine learning and deep learning algorithms analyzing vast datasets at speeds impossible for humans. These algorithms have exhibited remarkable accuracy in detecting diseases at early stages, such as cancer through image analysis or identifying abnormalities in radiological and pathological scans. The precision and speed at which AI processes medical data have not only improved diagnostic accuracy but also reduced the time required for these critical assessments, potentially saving countless lives.

One of AI's most profound impacts in healthcare is the creation of personalized treatment plans tailored to individual patients. By integrating a patient's medical history, genetic data, and lifestyle factors, AI-driven systems can generate treatment recommendations that optimize outcomes and minimize side effects. This approach, often referred to as precision medicine, represents a monumental shift from the one-size-fits-all model of the past, potentially revolutionizing the treatment of various diseases, including cancer, by targeting specific genetic mutations or biomarkers unique to each patient. Patients benefit from more effective treatments, while healthcare providers can make data-driven decisions to ensure better patient outcomes.

In addition to clinical applications, AI has made significant inroads in improving the administrative aspects of healthcare. The digitization of medical records, combined with AI's natural language processing capabilities, has streamlined the management of patient information, enabling quick access to critical data and reducing errors associated with manual record-keeping. AI algorithms are also adept at optimizing hospital operations, from scheduling patient appointments to billing and claims processing. These administrative efficiencies not only save time but also reduce costs, contributing to more accessible and affordable healthcare for patients.

Furthermore, AI has played a pivotal role in the rise of telemedicine and remote patient monitoring, especially in the context of global health crises such as the COVID-19 pandemic. Telemedicine, driven by AI-driven virtual consultations, has bridged geographical gaps and improved access to medical care, enabling patients to receive timely advice and treatment without the need for physical visits. Remote patient monitoring, facilitated by wearable health technology and AI algorithms, allows for continuous tracking of vital signs and disease progression, enabling early interventions and reducing the burden on healthcare facilities. This combination of telemedicine and remote monitoring has not only transformed healthcare delivery but has also proven crucial in managing public health emergencies.

In the realm of drug discovery and development, AI has accelerated traditionally laborious and expensive processes. Machine learning models analyze vast datasets to identify potential drug candidates, speeding up the initial stages of drug discovery. AI is particularly promising in drug repurposing, where existing drugs are reevaluated for new therapeutic uses, potentially bypassing years of research and clinical trials. Additionally, AI aids in clinical trial design by identifying suitable patient cohorts and predicting trial outcomes, thereby enhancing the efficiency and success rate of drug development. This innovation is poised to address the urgent need for novel treatments and therapies in various fields, including oncology and infectious diseases.

However, the widespread adoption of AI in healthcare does not come without its ethical considerations. The exponential growth in medical data, often sensitive and personal, raises concerns about data privacy and security. Ensuring that patient information remains confidential and protected from cyber threats becomes a paramount challenge. Moreover, AI algorithms are susceptible to bias, reflecting historical disparities present in healthcare data. Recognizing and mitigating algorithmic bias is essential to prevent the exacerbation of existing healthcare disparities based on race, ethnicity, or socioeconomic status. Striking the right balance between automation and human oversight in medical decision-making also poses ethical questions. While AI can augment clinical decision-making, there is a need to define the scope of AI's authority and ensure that human expertise remains integral in complex medical situations.

The integration of AI in healthcare is also reshaping the doctor-patient relationship. Patients now have access to AI-powered health apps and wearable devices, enabling them to monitor their health in real-time and actively engage in their own care. This shift towards patient-centric care empowers individuals to take control of their health and make informed decisions, potentially leading to better health outcomes. However, it also raises questions about the role of healthcare professionals in this new paradigm and the need for effective communication between AI systems, healthcare providers, and patients.

Looking forward, the future of healthcare with AI is both promising and complex. AI's potential to improve patient outcomes, increase cost-effectiveness, and revolutionize the healthcare experience is undeniable. However, the field continues to face challenges, including the need for robust regulatory frameworks that ensure patient safety and data privacy. Additionally, addressing the shortage of skilled AI professionals in healthcare and fostering interdisciplinary collaborations between clinicians and data scientists will be crucial for continued progress. As AI continues to evolve and become an integral part of healthcare, the industry must navigate these challenges to unlock its full potential and provide the best possible care to patients.

In conclusion, artificial intelligence has ushered in a new era for healthcare, fundamentally transforming how we diagnose, treat, and manage medical conditions. Its applications, from enhancing medical diagnoses and personalized treatment plans to streamlining administrative tasks and drug discovery, hold the promise of improving patient outcomes and making healthcare more accessible and efficient. However, the ethical considerations surrounding data privacy, algorithmic bias, and the evolving doctor-patient relationship necessitate careful navigation. As we venture into the future of healthcare with AI, a balanced approach that combines innovation, regulation, and collaboration will be essential to fully harness its transformative potential.

Artificial Intelligence (AI) has emerged as a groundbreaking tool in the field of Magnetic Resonance Imaging (MRI), revolutionizing the way medical professionals acquire, analyze, and interpret images of the human body. This 3000-token exploration delves into the multifaceted impact of AI in MRI, covering its applications in image enhancement, disease detection, image reconstruction, and the challenges and ethical considerations surrounding its adoption.

AI in Image Enhancement : AI algorithms have significantly improved image quality in MRI scans. Machine learning models can reduce noise, correct artifacts, and enhance image contrast, providing radiologists with clearer and more detailed images. This enhancement aids in the accurate diagnosis of various medical conditions, from brain tumors to musculoskeletal disorders.

Disease Detection and Diagnosis : AI's ability to analyze complex patterns within MRI images has transformed disease detection and diagnosis. Machine learning algorithms can detect subtle abnormalities, such as early-stage cancers, lesions, or neurological disorders, with high accuracy. This early detection can lead to timely interventions and improved patient outcomes.

Image Reconstruction: AI-powered image reconstruction techniques have expedited MRI procedures. Through deep learning algorithms, images can be reconstructed from fewer data points or faster acquisition times, reducing patient discomfort and scanner time. This innovation is especially crucial in pediatric and claustrophobic patients.

Real-time Image Analysis : AI enables real-time image analysis during MRI scans, allowing radiologists to adjust imaging parameters on the fly for optimal results. This dynamic approach ensures that the highest quality images are obtained, reducing the need for repeated scans and minimizing patient radiation exposure.

Quantitative Imaging : AI facilitates quantitative analysis of MRI data, extracting valuable information beyond what the human eye can perceive. It enables the measurement of tissue characteristics, such as volume, density, and perfusion, aiding in disease characterization, treatment planning, and monitoring.

Challenges and Considerations : Despite its promise, the integration of AI in MRI is not without challenges. The need for large, high-quality datasets for training AI models, data privacy concerns, and regulatory hurdles pose significant obstacles. Additionally, ensuring the reliability and interpretability of AI-generated results remains a priority to gain the trust of healthcare professionals.

"""

In [ ]:
from multiprocessing import Process, Manager
import numpy as np
import time

def invoke_endpoint(name, L):
    start_time = time.time()
    print(f"{name}:Calling endpoint")
    chat = llm.predict({"inputs": tokens_2k,
                        "parameters": {
                                    "max_new_tokens": 500,
                                    "min_new_tokens": 500,
                                    "do_sample": True,
                                    "temperature": 0.4,
                                    "return_full_text": False
                            }
                       }
                      )
    end_time = time.time()
    L.append(end_time-start_time)
    print(chat)

all_processes = []
with Manager() as manager:
    L = manager.list()
    for i in range(10):
        p = Process(target=invoke_endpoint, args=('Request'+str(i), L))
        p.start()
        all_processes.append(p)

    for p in all_processes:
        p.join()
        
    print("Latency (p90): ", np.percentile(L, 90))

## 4. Run inference and chat with the model

After our endpoint is deployed we can run inference on it. We will use the `predict` method from the `predictor` to run inference on our endpoint. We can inference with different parameters to impact the generation. Parameters can be defined as in the `parameters` attribute of the payload. As of today the TGI supports the following parameters:
* `temperature`: Controls randomness in the model. Lower values will make the model more deterministic and higher values will make the model more random. Default value is 1.0.
* `max_new_tokens`: The maximum number of tokens to generate. Default value is 20, max value is 512.
* `repetition_penalty`: Controls the likelihood of repetition, defaults to `null`.
* `seed`: The seed to use for random generation, default is `null`.
* `stop`: A list of tokens to stop the generation. The generation will stop when one of the tokens is generated.
* `top_k`: The number of highest probability vocabulary tokens to keep for top-k-filtering. Default value is `null`, which disables top-k-filtering.
* `top_p`: The cumulative probability of parameter highest probability vocabulary tokens to keep for nucleus sampling, default to `null`
* `do_sample`: Whether or not to use sampling ; use greedy decoding otherwise. Default value is `false`.
* `best_of`: Generate best_of sequences and return the one if the highest token logprobs, default to `null`.
* `details`: Whether or not to return details about the generation. Default value is `false`.
* `return_full_text`: Whether or not to return the full text or only the generated part. Default value is `false`.
* `truncate`: Whether or not to truncate the input to the maximum length of the model. Default value is `true`.
* `typical_p`: The typical probability of a token. Default value is `null`.
* `watermark`: The watermark to use for the generation. Default value is `false`.


## 5. Clean up

To clean up, we can delete the model and endpoint.


In [ ]:
llm.delete_model()
llm.delete_endpoint()